In [2]:
pip install qdrant-client groq sentence-transformers dspy-ai fastembed gradio --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.8/328.8 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [4]:
!pip install langchain langchain-google-genai PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.2/164.2 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.3/718.3 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 26.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.27.2
    Uninstalling protobuf-5.27.2:
      Successfully uninstalled protobuf-5.27.2
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.4
    Uninstalling google-ai-generativelanguage-0.6.4:
      Successfully uninstalled google-ai-generativelanguage-0.6.4
  Attempting uni

In [5]:
from qdrant_client import QdrantClient
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import dotenv

In [9]:
from google.colab import userdata
qdrant_api_key =userdata.get('qdrant_api_key')
qdrant_client = QdrantClient(
    url="https://77521ce6-077a-4d24-b77d-ff3eb040151b.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=qdrant_api_key,
)
print(qdrant_client.get_collections())

collections=[]


In [10]:
from PyPDF2 import PdfReader
def get_pdf_text(pdf_path):
    text = ""
    pdf_reader = PdfReader(pdf_path)
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

# Verify and correct the file path
pdf_path = "/content/data/dataset.pdf"

# Check if the file exists
text = get_pdf_text(pdf_path)

In [12]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

chunks = get_text_chunks(text)

In [14]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("BAAI/bge-large-en-v1.5", device='cpu')
vectors = model.encode(chunks)

In [15]:
vectors[0].shape

(1024,)

In [18]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
qdrant_client.recreate_collection(
    collection_name="dspy-application",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

<ipython-input-18-08307832fc2e>:3: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


True

In [19]:
qdrant_client.upload_collection(
    collection_name="dspy-application",
    ids=[i for i in range(len(chunks))],
    vectors=vectors,
    parallel=4,
    max_retries=3,
)

In [20]:
print(qdrant_client.get_collections())

collections=[CollectionDescription(name='dspy-application')]


In [21]:
from dspy.retrieve.qdrant_rm import QdrantRM
qdrant_retriever_model = QdrantRM("dspy-application", qdrant_client, k=3)

100%|██████████| 77.7M/77.7M [00:01<00:00, 67.8MiB/s]


In [22]:
import dspy
groq_api_key =userdata.get('groq_api_key')
llama3 = dspy.GROQ(model='llama3-8b-8192', api_key = groq_api_key )

In [23]:
dspy.settings.configure(rm=qdrant_retriever_model, lm=llama3)

In [37]:
class GenerateAnswer(dspy.Signature):
    """Answer questions with logical factoid answers."""

    context = dspy.InputField(desc="will contain chemistry related data")
    question = dspy.InputField()
    answer = dspy.OutputField(desc="a detailed answer")

In [38]:
def get_context(text):
    query_vector = model.encode(text)


    hits = qdrant_client.search(
        collection_name="dspy-application",
        query_vector=query_vector,
        limit=3  # Return 5 closest points
    )
    s=''
    for x in [chunks[i.id] for i in hits]:
        s = s + x
    return s

In [39]:
class RAG(dspy.Module):
    def __init__(self, num_passages=3):
        super().__init__()


        self.retrieve = dspy.Retrieve(k=num_passages)
        self.generate_answer = dspy.ChainOfThought(GenerateAnswer)


    def forward(self, question):
        context = get_context(question)
        prediction = self.generate_answer(context=context, question=question)
        return dspy.Prediction(context=context, answer=prediction.answer)

In [40]:
rag = RAG()
def respond(query):
    response = rag(query)
    return response.answer

In [43]:
rag = RAG()
def respond(query, chat_history=None): # Add chat_history with a default value
    response = rag(query)
    return response.answer

answer  = respond("0") # Call without chat_history, using the default
print(answer)

INFO:backoff:Backing off request(...) for 0.1s (groq.BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


Backing off 0.1 seconds after 1 tries calling function <function GroqLM.request at 0x7e415dff5f30> with kwargs {}


INFO:backoff:Backing off request(...) for 1.2s (groq.BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


Backing off 1.2 seconds after 2 tries calling function <function GroqLM.request at 0x7e415dff5f30> with kwargs {}


INFO:backoff:Backing off request(...) for 1.7s (groq.BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


...

INFO:backoff:Backing off request(...) for 71.3s (groq.BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


Backing off 71.3 seconds after 8 tries calling function <function GroqLM.request at 0x7e415dff5f30> with kwargs {}


INFO:backoff:Backing off request(...) for 201.9s (groq.BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}})


Backing off 201.9 seconds after 9 tries calling function <function GroqLM.request at 0x7e415dff5f30> with kwargs {}


KeyboardInterrupt: 